## 1. Setup and Imports

In [ ]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.insert(0, '/workspaces/object-detection/car-detection/src')

from utils import load_image, save_image, draw_boxes, get_image_files
from predict_model import CarDetector

print("Imports successful!")
print(f"CUDA available: {torch.cuda.is_available() if 'torch' in dir() else 'check below'}")

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Data Preparation

### Load and Explore Dataset

In [ ]:
# Set paths
PROJECT_ROOT = '/workspaces/object-detection/car-detection'
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
CONFIG_PATH = os.path.join(PROJECT_ROOT, 'config/car_data.yaml')

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Config path: {CONFIG_PATH}")

# Check if data exists
if os.path.exists(DATA_DIR):
    print("\nData directory structure:")
    for root, dirs, files in os.walk(DATA_DIR):
        level = root.replace(DATA_DIR, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files[:3]:  # Show first 3 files
            print(f'{subindent}{file}')
else:
    print("Data directory not found. Create sample data or use prepare_data.py")

### Organize Data from Source

If you have a folder with images and labels, organize them using this code:

In [ ]:
# Example: Organize data from a source directory
# Uncomment and modify the path if you have raw data

# from prepare_data import organize_images_and_labels
# 
# source_dir = '/path/to/raw/images'  # Change this
# output_dir = os.path.join(PROJECT_ROOT, 'data')
# 
# train, val, test = organize_images_and_labels(
#     source_dir,
#     output_dir,
#     train_ratio=0.8,
#     val_ratio=0.1
# )
#
# print(f"Train: {train}, Val: {val}, Test: {test}")

## 3. Model Training

### Download and Initialize Model

In [ ]:
# Load YOLOv8 model
# Available models: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
model_name = 'yolov8n.pt'  # nano - fastest
# model_name = 'yolov8m.pt'  # medium - balanced
# model_name = 'yolov8l.pt'  # large - more accurate

print(f"Loading model: {model_name}")
model = YOLO(model_name)
print(f"Model loaded successfully!")

### Configure Training Parameters

In [ ]:
# Training configuration
config = {
    'data': CONFIG_PATH,
    'epochs': 10,  # Small number for notebook demo
    'imgsz': 640,
    'batch': 16,  # Adjust based on GPU memory
    'device': 0,  # GPU device (0 for first GPU, -1 for CPU)
    'project': os.path.join(PROJECT_ROOT, 'runs/detect'),
    'name': 'car_detection',
    'patience': 10,
    'save': True,
    'save_period': 5,
    'verbose': True,
}

print("Training Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

### Start Training

**Note**: Training requires a properly formatted dataset with images and YOLO format labels. If you don't have data yet, skip this cell.

In [ ]:
# Train model
# Uncomment to train (requires dataset with proper structure)

# if os.path.exists(CONFIG_PATH):
#     print("Starting training...")
#     results = model.train(**config)
#     print(f"\nTraining completed!")
#     print(f"Best model saved to: {config['project']}/{config['name']}/weights/best.pt")
# else:
#     print(f"Config file not found: {CONFIG_PATH}")

## 4. Model Inference (Prediction)

### Download a Sample Image for Testing

In [ ]:
# For demonstration, we'll use YOLOv8's pretrained model to detect cars
# In production, this would be your trained model

# Create a sample image path for demo (you would use your own images)
sample_image_dir = os.path.join(PROJECT_ROOT, 'samples')
os.makedirs(sample_image_dir, exist_ok=True)

print(f"Sample images directory: {sample_image_dir}")
print("Place your test images here for inference.")

### Initialize Detector and Make Predictions

In [ ]:
# Example: Predict on a single image
# This example uses the pretrained YOLOv8 model

def predict_on_image(model, image_path, conf_threshold=0.5):
    """
    Make prediction on a single image.
    
    Args:
        model: YOLO model
        image_path: Path to image file
        conf_threshold: Confidence threshold
    """
    if not os.path.exists(image_path):
        print(f"Image not found: {image_path}")
        return None
    
    # Run inference
    results = model(image_path, conf=conf_threshold, verbose=False)
    
    # Get results
    if len(results) > 0:
        result = results[0]
        detections = []
        
        if result.boxes is not None:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                conf = box.conf[0].item()
                cls_id = int(box.cls[0].item())
                cls_name = result.names[cls_id]
                
                detection = {
                    'box': [x1, y1, x2, y2],
                    'confidence': conf,
                    'class': cls_name
                }
                detections.append(detection)
        
        return result, detections
    
    return None, []

print("Prediction function defined!")

In [ ]:
# Example: Predict and visualize
# Replace with your image path

# example_image = os.path.join(sample_image_dir, 'your_image.jpg')
# 
# if os.path.exists(example_image):
#     result, detections = predict_on_image(model, example_image, conf_threshold=0.5)
#     
#     if detections:
#         print(f"\nDetected {len(detections)} objects:")
#         for i, det in enumerate(detections, 1):
#             print(f"  {i}. {det['class']}: confidence={det['confidence']:.3f}")
#     else:
#         print("No objects detected")
# else:
#     print(f"Please place a test image at: {example_image}")

### Batch Prediction

In [ ]:
def batch_predict(model, image_dir, conf_threshold=0.5):
    """
    Run predictions on all images in a directory.
    
    Args:
        model: YOLO model
        image_dir: Directory containing images
        conf_threshold: Confidence threshold
    """
    image_files = get_image_files(image_dir)
    
    if not image_files:
        print(f"No images found in {image_dir}")
        return {}
    
    results = {}
    print(f"Processing {len(image_files)} images...")
    
    for image_path in image_files:
        result, detections = predict_on_image(model, image_path, conf_threshold)
        results[image_path] = {
            'result': result,
            'detections': detections,
            'count': len(detections)
        }
    
    return results

print("Batch prediction function defined!")

## 5. Model Evaluation

### Validate Model Performance

In [ ]:
# Validate on validation set
# Requires trained model and validation dataset

# Uncomment to evaluate
# if os.path.exists(CONFIG_PATH):
#     print("Validating model...")
#     metrics = model.val(data=CONFIG_PATH)
#     
#     print(f"\nValidation Results:")
#     print(f"mAP50: {metrics.box.map50:.4f}")
#     print(f"mAP50-95: {metrics.box.map:.4f}")
# else:
#     print("Config file not found for validation")

## 6. Visualization and Analysis

### Plot Detection Results

In [ ]:
def visualize_detections(image_path, detections, figsize=(12, 8)):
    """
    Visualize detections on image.
    
    Args:
        image_path: Path to image
        detections: List of detections
        figsize: Figure size
    """
    # Load image
    image = load_image(image_path)
    if image is None:
        print(f"Could not load image: {image_path}")
        return
    
    # Draw boxes
    annotated = draw_boxes(image, detections)
    
    # Display
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(annotated)
    ax.set_title(f'Detections: {len(detections)} objects')
    ax.axis('off')
    plt.tight_layout()
    plt.show()

print("Visualization function defined!")

### Summary Statistics

In [ ]:
def print_detection_summary(results_dict):
    """
    Print summary of detections.
    
    Args:
        results_dict: Dictionary of batch prediction results
    """
    if not results_dict:
        print("No results to summarize")
        return
    
    total_images = len(results_dict)
    total_detections = sum(r['count'] for r in results_dict.values())
    avg_detections = total_detections / total_images if total_images > 0 else 0
    
    print("\n" + "="*50)
    print("DETECTION SUMMARY")
    print("="*50)
    print(f"Total images: {total_images}")
    print(f"Total detections: {total_detections}")
    print(f"Average detections per image: {avg_detections:.2f}")
    print("="*50)

print("Summary function defined!")

## 7. Model Management

### Save Trained Model

In [ ]:
# Save model
# Uncomment after training

# model_save_path = os.path.join(PROJECT_ROOT, 'models/car_detector.pt')
# model.save(model_save_path)
# print(f"Model saved to: {model_save_path}")

### Load Saved Model

In [ ]:
# Load previously trained model
# model_path = os.path.join(PROJECT_ROOT, 'models/car_detector.pt')
# 
# if os.path.exists(model_path):
#     model = YOLO(model_path)
#     print(f"Model loaded: {model_path}")
# else:
#     print(f"Model not found: {model_path}")

## 8. Export Model

### Export to Different Formats

In [ ]:
# Export model to different formats
# Uncomment to export after training

# export_formats = ['onnx', 'torchscript', 'pb', 'tflite']
# 
# for fmt in export_formats:
#     try:
#         export_path = model.export(format=fmt)
#         print(f"Exported to {fmt}: {export_path}")
#     except Exception as e:
#         print(f"Failed to export to {fmt}: {e}")

## 9. Quick Reference

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════════════╗
║              CAR DETECTION PROJECT - QUICK REFERENCE                   ║
╚════════════════════════════════════════════════════════════════════════╝

PROJECT STRUCTURE:
  📁 /workspaces/object-detection/car-detection/
    ├── 📁 data/          - Training data (train/val/test)
    ├── 📁 src/           - Python modules
    ├── 📁 config/        - Configuration files
    ├── 📁 docs/          - Documentation
    ├── 📁 models/        - Trained models
    ├── 📁 runs/          - Training outputs
    └── 📄 README.md      - Main documentation

COMMAND LINE TOOLS:
  # Prepare data
  python src/prepare_data.py --source /path/images --output data
  
  # Train model
  python src/train_model.py --config config/car_data.yaml --epochs 100
  
  # Make predictions
  python src/predict_model.py --model best.pt --source image.jpg
  
  # Evaluate model
  python src/evaluate.py --model best.pt --data config/car_data.yaml

PYTHON API:
  from src.predict_model import CarDetector
  detector = CarDetector('model.pt')
  annotated, detections = detector.detect(image)

DOCUMENTATION:
  📘 README.md              - Quick start
  📘 USAGE.md               - Detailed usage guide
  📘 ARCHITECTURE.md        - System design
  📘 CONFIG_REFERENCE.md    - Configuration options
  📘 DEVELOPMENT.md         - Development guide

NEXT STEPS:
  1. Prepare your dataset using prepare_data.py
  2. Train model with train_model.py
  3. Make predictions with predict_model.py
  4. Evaluate performance with evaluate.py
""")